In [1]:
import sys
import pickle
import json
import re
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [2]:
def feature_transform(train_file, tags, chunk_tags):
    
    tags_len = len(tags)
    chunk_tags_len = len(chunk_tags)    
    row_idx = []
    column_idx = []
    data = []
    y = []
    ctr = 0
    feature_length = tags_len + chunk_tags_len # Per head

    f = open(train_file)

    for line in f:
            if(line.rstrip()):
                line = re.sub("\s+"," ",line)
                line1 = line.split(";")

                a1 = line1[0].split(" ")
                a2 = line1[1].split(" ")
                a3 = line1[2].split(" ")
                a4 = line1[3].strip()

                if a3[1] != "U":
                    
                    ctr += 1

                    if(a1[0] == "H"):
                        column_idx.append(tags.index(a1[4]))
                        column_idx.append(chunk_tags.index(a1[3]) + tags_len)

                    elif(a1[0] == "ROOT"):
                        column_idx.append(tags.index("ROOT"))
                        column_idx.append(chunk_tags.index("ROOT") + tags_len)

                    row_idx += [ctr-1]*4
                    data += [1] * 4
                    column_idx.append(feature_length + tags.index(a2[5]))
                    column_idx.append(feature_length + chunk_tags.index(a2[4]) + tags_len)

                    y.append(a4)

    f.close()

    X = csr_matrix((data, (row_idx, column_idx)), shape=(ctr,2*(tags_len+chunk_tags_len)))
    
    
    return X, y

In [3]:
listfile = "data_tokens.json"
f = open(listfile)
data = json.load(f)
f.close()

tags = data["tags"]
chunk_tags = data["chunk_tags"]

train_file = 'training_data.txt'
test_file = "testing_data.txt"

X_train, y_train = feature_transform(train_file, tags, chunk_tags)
X_test, y_test = feature_transform(test_file, tags, chunk_tags)


model = LinearSVC()
model.fit(X_train, y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [4]:
print(classification_report(y_train, pred_train))

/home/ayan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         K2U       0.00      0.00      0.00         1
        ROOT       1.00      1.00      1.00     11145
       UNDEF       0.00      0.00      0.00         1
         adv       0.71      0.55      0.62      1325
        ccof       0.81      0.90      0.85     12223
         enm       0.00      0.00      0.00         2
      fragof       0.50      0.22      0.31         9
       jjmod       0.91      0.95      0.93       716
         jk1       0.00      0.00      0.00        10
          k1       0.31      0.89      0.46     13000
         k1s       0.32      0.05      0.09      1487
         k1u       0.00      0.00      0.00        46
          k2       0.43      0.51      0.47     10234
         k2g       0.00      0.00      0.00        31
         k2p       0.00      0.00      0.00       387
         k2s       0.00      0.00      0.00       296
         k2u       0.00      0.00      0.00        34
          k3       0.00    

In [5]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

         K7a       0.00      0.00      0.00         1
        ROOT       1.00      1.00      1.00      1432
         adv       0.87      0.66      0.75       147
        ccof       0.81      0.90      0.86      1558
      fragof       0.00      0.00      0.00         0
       jjmod       0.91      0.96      0.93        80
          k1       0.30      0.91      0.45      1584
         k1s       0.43      0.09      0.14       210
         k1u       0.00      0.00      0.00         9
          k2       0.49      0.51      0.50      1369
         k2p       0.00      0.00      0.00        40
         k2s       0.00      0.00      0.00        29
         k2u       0.00      0.00      0.00         1
          k3       0.00      0.00      0.00        54
          k4       0.00      0.00      0.00       197
         k4a       0.00      0.00      0.00        17
          k5       0.00      0.00      0.00       125
          k7       0.00    

/home/ayan/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
